In [ ]:
# **RNA-Seq Analysis Pipeline: From FASTA Files to Combined Gene Expression Matrix**
# Python dependencies
pip install pandas
# R dependencies
install.packages("dplyr")
## Step 1: Setup Kallisto with terminal and follow their website for your computer system and Index the Reference Genome (Python)

!conda install -c bioconda kallisto

# Import necessary libraries
import os
import subprocess

# Set the paths to your input files and directories
reference_genome_fasta = "path_to_reference_genome.fasta"  # Replace with your actual path
kallisto_index = "path_to_save_index/kallisto_index.idx"

# Step 1: Index the reference genome
# This command will create an index file from the reference genome using Kallisto
kallisto_command = f"kallisto index -i {kallisto_index} {reference_genome_fasta}"



# Run the command
subprocess.run(kallisto_command, shell=True)

print("Reference genome has been indexed successfully!")


In [ ]:
# Step 2: Quantify RNA-seq Data using Kallisto (Python)

# Set up your input FASTA files (paired-end or single-end)
input_dir = "path_to_fasta_files"
output_dir = "path_to_output_directory"
kallisto_index = "path_to_save_index/kallisto_index.idx"

# Get a list of FASTA files in the input directory
fasta_files = [f for f in os.listdir(input_dir) if f.endswith(".fastq")]

# Align FASTA files with Kallisto (paired-end example)
for sample in fasta_files:
    fastq_file_1 = os.path.join(input_dir, f"{sample}_1.fastq")  # Paired-end read 1
    fastq_file_2 = os.path.join(input_dir, f"{sample}_2.fastq")  # Paired-end read 2
    output_sample_dir = os.path.join(output_dir, sample)
    
    # Create an output directory for each sample
    if not os.path.exists(output_sample_dir):
        os.makedirs(output_sample_dir)
    
    # Run kallisto quant
    kallisto_quant_cmd = f"kallisto quant -i {kallisto_index} -o {output_sample_dir} {fastq_file_1} {fastq_file_2}"
    subprocess.run(kallisto_quant_cmd, shell=True)

print("Kallisto quantification completed for all samples!")


In [ ]:
import os
import subprocess
# Step 3: Combine Kallisto Output into a Single Matrix (Python)

# RUN WITH MULTIPLE CORES BECAUSE PROCESS IS SLOW FOR MANY FILES
# Define the number of threads (cores) to use
num_threads = 8  # Adjust based on your system

# Define the directory where the FASTA files are located
input_dir = "path_to_fasta_files"  # Update this path
output_dir = "path_to_output_directory"  # Update this path
kallisto_index = "path_to_save_index/kallisto_index.idx"  # Update this path

# List the samples (paired-end reads)
fasta_files = ["sample1", "sample2", "sample3"]  # Replace with your actual sample names

# Align each sample with Kallisto using multiple cores
for sample in fasta_files:
    fastq_file_1 = os.path.join(input_dir, f"{sample}_1.fastq")  # Paired-end read 1
    fastq_file_2 = os.path.join(input_dir, f"{sample}_2.fastq")  # Paired-end read 2
    output_sample_dir = os.path.join(output_dir, sample)
    
    # Create output directory for each sample
    if not os.path.exists(output_sample_dir):
        os.makedirs(output_sample_dir)
    
    # Kallisto quantification command with multiple threads
    kallisto_command = f"kallisto quant -i {kallisto_index} -o {output_sample_dir} -t {num_threads} {fastq_file_1} {fastq_file_2}"
    
    # Run the command
    subprocess.run(kallisto_command, shell=True)

print(f"Kallisto quantification completed for all samples using {num_threads} threads!")


In [ ]:
import pandas as pd
# Step 3: Combine Kallisto Output into a Single Matrix (Python)

# Define the directories
input_dir = "path_to_output_directory"
output_file = "path_to_save/gene_expression_matrix.csv"
sample_ids = ["Sample1", "Sample2", "Sample3"]  # Replace with your actual sample IDs

# Initialize an empty dataframe to hold all data
combined_df = pd.DataFrame()

# Loop through each sample and read in its 'abundance.tsv' file from Kallisto output
for sample_id in sample_ids:
    file_path = os.path.join(input_dir, sample_id, "abundance.tsv")
    sample_df = pd.read_csv(file_path, sep="\t")
    
    # Extract 'target_id' and 'est_counts'
    sample_df = sample_df[['target_id', 'est_counts']]
    
    # Simplify target_id to just gene ID
    sample_df['target_id'] = sample_df['target_id'].apply(lambda x: x.split('|')[0])
    
    # Rename columns to reflect sample ID
    sample_df.columns = ['Gene_ID', sample_id]
    
    # Merge into the combined dataframe
    if combined_df.empty:
        combined_df = sample_df
    else:
        combined_df = pd.merge(combined_df, sample_df, on='Gene_ID')

# Save the combined raw counts matrix
combined_df.to_csv(output_file, index=False)

print(f"Combined gene expression matrix saved to {output_file}")


In [ ]:
# Step 4: Mapping Gene IDs to Gene Names (R), can be done in python also if you want 

# Load necessary libraries
library(dplyr)

# Set the working directory
setwd("C:/Users/Amanda/Documents/Bulk RNAseq Analysis/clean_reads")

# Load the gene expression matrix generated from the previous step
expression_matrix <- read.csv("C:/Users/Amanda/Documents/Bulk RNAseq Analysis/clean_reads/gene_expression_matrix.csv", header = TRUE)

# BioMart information:
# When running RNA-seq analysis with Kallisto, the output typically contains Ensembl transcript IDs (e.g., ENST00000456328).
# To map these IDs to gene names, we use BioMart, which provides a downloadable file that maps transcript IDs to gene names.
# The BioMart file was downloaded with relevant gene annotations (e.g., Ensembl transcript stable IDs and gene names).
# Ensure that the column names match the Ensembl identifiers used in Kallisto output.

# Load the BioMart file (this file contains the mappings of ENST IDs to gene names)
biomart_data <- read.delim("C:/Users/Amanda/Documents/Bulk RNAseq Analysis/clean_reads/biomart.txt", header = TRUE, sep = ",")

# Merge the expression matrix with the BioMart data to replace ENST IDs with gene names
# Make sure to check that "Transcript.stable.ID.version" in the BioMart file corresponds to the Kallisto IDs
merged_data <- merge(expression_matrix, biomart_data[, c("Transcript.stable.ID.version", "Gene.name")], 
                     by.x = "Gene_ID", by.y = "Transcript.stable.ID.version", all.x = TRUE)

# Replace ENST IDs with gene names where available
merged_data$Gene_ID <- ifelse(is.na(merged_data$Gene.name), merged_data$Gene_ID, merged_data$Gene.name)

# Remove extra column
merged_data <- merged_data[, -which(names(merged_data) == "Gene.name")]

# Save the final expression matrix with gene names
write.csv(merged_data, "APLNR_gene_names.csv", row.names = FALSE)

print("Gene names added and matrix saved successfully!")
